# LangChain Basics

LangChain is a framework designed to simplify the development of applications involving large language models (LLMs). LangChain allows you to build powerful, context-aware applications by leveraging its tools for chaining LLMs.

In [ ]:
!pip install -q langchain==0.2.2
!pip install -q langchain_community==0.2.3
!pip install -q langchain-openai==0.1.8

## Prerequisite: Generate OpenAI key

Generate key from: https://platform.openai.com/api-keys

In [ ]:
import os
from google.colab import userdata

os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")

## Call OpenAI using Langchain Model

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage
from langchain_core.output_parsers import StrOutputParser

model = ChatOpenAI(model="gpt-3.5-turbo")

messages = [
    HumanMessage(content="Hello!")
]
m = model.invoke(messages)
# m.content: message from LLM

parser = StrOutputParser()
parser.invoke(m)

In [ ]:
chain = model | parser # write process as chain
chain.invoke(messages) # same result

## Building Improve Grammar App using Langchain PromptTemplate

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

system_template = "You will be provided with statements, and your task is to convert them to standard English."
style_prompts = {
    "default": "",
    "academic": "Ensure that the language used is appropriate for an academic research publication.",
    "ielts": "Using fancy words. Ensure that the language used meets the standards required for an IELTS score of 8.0.",
    "informal": "Make it informal like talking to a friend."
}

user_text = "Hello again mine frinnds!"
user_style = "ielts"

In [ ]:
prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system", system_template + " {style}"),
        ("user", "{text}")
    ]
)

chain = prompt_template | model | parser

In [ ]:
def improve_grammar(text, style="default"):
    style_text = style_prompts[style]

    return chain.invoke({"text": text, "style": style_text})

improve_grammar(user_text, user_style)

## Including Chat History

In [ ]:
model = ChatOpenAI(model="gpt-3.5-turbo")
messages = [
    # include history
    HumanMessage(content="Hello my name is Oat!"),
    AIMessage(content="Hello Oat, nice to meet you! How can I assist you today?"),
    HumanMessage(content="What's my name?")
]

model.invoke(messages)

In [ ]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

with_message_history = RunnableWithMessageHistory(model, get_session_history) # wrap model object
config = {"configurable": {"session_id": "chat-01"}} # config session id

response = with_message_history.invoke(
    [
        HumanMessage(content="Hi! I'm Oat!")
    ],
    config=config,
)

print(store)

## Personalize Chatbot

In [ ]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt_template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant named \"Arise\"",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt_template | model
config = {"configurable": {"session_id": "chat-01"}}
store = {}

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

with_message_history = RunnableWithMessageHistory(chain, get_session_history)

response = with_message_history.invoke(
    [HumanMessage(content="Hi! I'm Oat!")],
    config=config,
)

print(response.content)

In [ ]:
response = with_message_history.invoke(
    [HumanMessage(content="What's your name?")],
    config=config,
)

print(response.content)